In [1]:
import sys
sys.path.append('..')

from analyse.prelude import *

In [2]:
x_trials, x_meta = load_files(["x-axis"], pendulum=True)
#plot_trials_w(x_trials, x_meta)

[INFO] Processing '../moi/physical/x-axis/Gyroscope rotation rate 2025-03-07 14-02-08/Raw Data.csv' ...
[INFO] Processing '../moi/physical/x-axis/Gyroscope rotation rate 2025-03-07 14-40-38/Raw Data.csv' ...
[INFO] -> discarding segment 1
[INFO] Found 22 trials in 2 files.


In [3]:
Tx, dTx = zero_method(x_trials, 'Gyroscope x (rad/s)')
print(f'Tx = {Tx:.3f} ± {dTx:.3f}, rel. unc. = {100*dTx/Tx:.3f}%')

[INFO] finding period based on 158 samples ...
Tx = 1.436 ± 0.039, rel. unc. = 2.698%


In [4]:
z_trials, z_meta = load_files(["z-axis"], pendulum=True)
z_trials, z_meta = z_trials[:16]+z_trials[18:], z_meta[:16]+z_meta[18:] # fix glitch in file processing later?

[INFO] Processing '../moi/physical/z-axis/Gyroscope rotation rate 2025-03-07 15-01-01/Raw Data.csv' ...
[INFO] Found 23 trials in 1 files.


In [5]:
Tz, dTz = zero_method(z_trials, 'Gyroscope z (rad/s)')
print(f'Tz = {Tz:.3f} ± {dTz:.3f}, rel. unc. = {100*dTz/Tz:.3f}%')

[INFO] finding period based on 160 samples ...
Tz = 1.450 ± 0.023, rel. unc. = 1.583%


In [22]:
g = 9.809    # m/s^2

r_cm = 0.413 # distance from top of racket to COM in meters
dr_cm = 0.02 / 4 # uncertainty in r_cm (based off 2cm 95% CI)

# we assume that uncertainty in the mass is negligible 
# compared to the other uncertainties involved
M_racquet = 0.285 # mass of racquet [kg]
M_phone = 0.17228 # mass of phone [kg]
M = M_racquet + M_phone

# equation for a physical pendulum:
# I = m g L T^2 / (4 pi^2)
Ix_butt = M * g * r_cm * Tx**2 / (4*np.pi**2)
Iz_butt = M * g * r_cm * Tz**2 / (4*np.pi**2)

# partial I / partial r_cm
Ix_partial_r = M * g * Tx**2 / (4*np.pi**2)
Iz_partial_r = M * g * Tz**2 / (4*np.pi**2)

# partial I / partial T
Ix_partial_T = 2 * M * g * r_cm * Tx / (4*np.pi**2)
Iz_partial_T = 2 * M * g * r_cm * Tz / (4*np.pi**2)

# propagate uncertainties
dI_x_butt = np.sqrt((Ix_partial_r * dr_cm)**2 + (Ix_partial_T * dTx)**2)
dI_z_butt = np.sqrt((Iz_partial_r * dr_cm)**2 + (Iz_partial_T * dTz)**2)

print(f'Ix_butt = ({Ix_butt:.5e} ± {dI_x_butt:.5e}) kg m^2, rel. unc. {100*dI_x_butt/Ix_butt:.3f}%')
print(f'Iz_butt = ({Iz_butt:.5e} ± {dI_z_butt:.5e}) kg m^2, rel. unc. {100*dI_z_butt/Iz_butt:.3f}%')
print()

# parallel axis theorem adjustment
delta_I = -M*r_cm**2
d_delta_I = 2*M*r_cm*dr_cm

print(f'parallel-axis adjustment:')
print(f'delta_I = ({delta_I:.5e} ± {d_delta_I:.5e}) kg m^2, rel. unc. {100*d_delta_I/abs(delta_I):.3f}%')
print()

Ix = Ix_butt + delta_I
Iz = Iz_butt + delta_I

dIx = np.sqrt(dI_x_butt**2 + d_delta_I**2)
dIz = np.sqrt(dI_z_butt**2 + d_delta_I**2)

print(f"I_x = ({Ix:.5e} ± {dIx:.5e}) kg m^2, rel. unc. {100*dIx/Ix:.3f}%")
print(f"I_z = ({Iz:.5e} ± {dIx:.5e}) kg m^2, rel. unc. {100*dIz/Iz:.3f}%")

Ix_butt = (9.68134e-02 ± 5.35376e-03) kg m^2, rel. unc. 5.530%
Iz_butt = (9.87011e-02 ± 3.34485e-03) kg m^2, rel. unc. 3.389%

parallel-axis adjustment:
delta_I = (-7.79978e-02 ± 1.88857e-03) kg m^2, rel. unc. 2.421%

I_x = (1.88157e-02 ± 5.67710e-03) kg m^2, rel. unc. 30.172%
I_z = (2.07033e-02 ± 5.67710e-03) kg m^2, rel. unc. 18.553%
